# Part 08: Visualize seattle street groups by type
michael babb  
2025 02 18

In [1]:
# standard
import os

In [2]:
# external
import geopandas as gpd
import matplotlib as mpl
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [3]:
# custom
import run_constants as rc
from utils import get_sort_order

# load the working seattle street network data

In [4]:
fpn = os.path.join(rc.OUTPUT_FILE_PATH, rc.S03_SND_WORKING_IN_FILE_NAME)

In [5]:
gdf = gpd.read_file(filename = fpn)

In [6]:
gdf.shape

(25773, 39)

In [7]:
gdf.columns

Index(['f_intr_id', 't_intr_id', 'snd_id', 'snd_feacode', 'citycode',
       'stname_id', 'st_code', 'arterial_code', 'segment_type', 'agency_code',
       'access_code', 'divided_code', 'structure_type', 'legalloc_code',
       'vehicle_use_code', 'gis_seg_length', 'l_adrs_from', 'l_adrs_to',
       'r_adrs_from', 'r_adrs_to', 'ord_pre_dir', 'ord_street_name',
       'ord_street_type', 'ord_suf_dir', 'ord_stname_concat', 'l_city',
       'l_state', 'l_zip', 'r_city', 'r_state', 'r_zip', 'sndseg_update',
       'compkey', 'comptype', 'unitid', 'unitid2', 'city_sector',
       'ord_stname_unique', 'geometry'],
      dtype='object')

In [8]:
gdf['ord_stname_concat'].unique().shape

(2455,)

In [9]:
gdf['ord_stname_unique'].unique().shape

(2457,)

In [10]:
gdf['city_sector'].unique()

array(['NW', 'N', 'NE', 'CNTR', 'E', 'W', 'S', 'SW'], dtype=object)

## load the manually created street group data
These data were created in Step 6a, but we'll bring them into the Step 3 workflow.

In [11]:
fpn = os.path.join(rc.INPUT_FILE_PATH, rc.S03_STREET_GROUP_IN_FILE_NAME)

In [12]:
# ['sort_order', 'ord_street_name', 'ord_street_type', 'city_sector']
sg_df = pd.read_excel(io = fpn, index_col=[0, 1, 2, 3])
sg_df = sg_df.reset_index()

In [13]:
# remove some columns
drop_cols = ['sort_order', 'progress', 'count']
sg_df = sg_df.drop(labels = drop_cols, axis = 1)


In [14]:
test_join = pd.merge(left = gdf, right = sg_df, how = 'outer', indicator=True)
test_join['_merge'].value_counts()

_merge
both          19032
left_only      6741
right_only        0
Name: count, dtype: int64

In [15]:
gdf = test_join.drop(labels = ['_merge'], axis = 1)

In [16]:
gdf['group_id'] = gdf['group_id'].fillna(1).astype(int)
gdf.shape

(25773, 40)

# create an id across street groups

In [17]:
gdf.head()

,f_intr_id,t_intr_id,snd_id,snd_feacode,citycode,stname_id,st_code,arterial_code,segment_type,agency_code,...,r_zip,sndseg_update,compkey,comptype,unitid,unitid2,city_sector,ord_stname_unique,geometry,group_id
0,3836,3893,4787,5,1,1955,0,1,1,1,...,98177,2011-09-09 00:00:00+00:00,18112,68,14050,0040,NW,NW 100TH PL,"LINESTRING (-122.36206 47.7035, -122.36342 47....",1
1,3893,3906,4802,5,1,1955,0,1,1,1,...,98177,2011-09-09 00:00:00+00:00,18113,68,14050,0043,NW,NW 100TH PL,"LINESTRING (-122.36342 47.70275, -122.36386 47...",1
2,3906,3973,4864,5,1,1955,0,1,1,1,...,98177,2019-08-13 14:00:50+00:00,18114,68,14050,0070,NW,NW 100TH PL,"LINESTRING (-122.36386 47.70251, -122.36535 47...",1
3,48444,48443,49871,1,0,3767,0,0,1,1,...,98146,2022-07-19 16:58:07+00:00,0,0,0,0,SW,SW 100TH PL,"LINESTRING (-122.3403 47.51334, -122.34036 47....",2
4,3988,3991,4888,1,1,1465,0,0,1,1,...,98133,2005-05-04 00:00:00+00:00,14959,68,11695,0010,N,N 100TH ST,"LINESTRING (-122.35809 47.70148, -122.35677 47...",1


In [18]:
gdf.columns

Index(['f_intr_id', 't_intr_id', 'snd_id', 'snd_feacode', 'citycode',
       'stname_id', 'st_code', 'arterial_code', 'segment_type', 'agency_code',
       'access_code', 'divided_code', 'structure_type', 'legalloc_code',
       'vehicle_use_code', 'gis_seg_length', 'l_adrs_from', 'l_adrs_to',
       'r_adrs_from', 'r_adrs_to', 'ord_pre_dir', 'ord_street_name',
       'ord_street_type', 'ord_suf_dir', 'ord_stname_concat', 'l_city',
       'l_state', 'l_zip', 'r_city', 'r_state', 'r_zip', 'sndseg_update',
       'compkey', 'comptype', 'unitid', 'unitid2', 'city_sector',
       'ord_stname_unique', 'geometry', 'group_id'],
      dtype='object')

In [19]:
col_names = ['ord_street_name', 'ord_street_type', 'ord_stname_concat',
             'ord_stname_unique', 'city_sector', 'group_id']
id_df = gdf[col_names].drop_duplicates()

In [20]:
id_df['ord_stname_type'] = id_df['ord_street_name'] + ' ' + id_df['ord_street_type']

In [21]:
id_df['sort_order'] = id_df['ord_street_name'].map(get_sort_order)

In [22]:
id_df = id_df.sort_values(by = ['sort_order', 'ord_street_type', 'city_sector'])

In [23]:
id_df.head()

,ord_street_name,ord_street_type,ord_stname_concat,ord_stname_unique,city_sector,group_id,ord_stname_type,sort_order
3387,1ST,AVE,1ST AVE,1ST AVE CNTR,CNTR,2,1ST AVE,001
3412,1ST,AVE,1ST AVE N,1ST AVE N,N,2,1ST AVE,001
3449,1ST,AVE,1ST AVE NE,1ST AVE NE,NE,2,1ST AVE,001
3504,1ST,AVE,1ST AVE NW,1ST AVE NW,NW,1,1ST AVE,001
3578,1ST,AVE,1ST AVE S,1ST AVE S,S,2,1ST AVE,001


In [24]:
id_df['ord_stname_unique'].unique().shape

(2457,)

In [25]:
# let's get the groups! can we aggregate this?

In [26]:
col_names = ['sort_order', 'ord_street_name', 'ord_street_type', 'ord_stname_type', 'group_id', 'city_sector']
test_agg = id_df[col_names].groupby(col_names[:-1]).agg(city_sector_group = ('city_sector', lambda x: '_'.join(sorted(set(x)))),
                                                        n_groups = ('city_sector', lambda x: len(set(x)))).reset_index()

In [27]:
test_agg.head()

,sort_order,ord_street_name,ord_street_type,ord_stname_type,group_id,city_sector_group,n_groups
0,001,1ST,AVE,1ST AVE,1,NW_SW_W,3
1,001,1ST,AVE,1ST AVE,2,CNTR_N_NE_S,4
2,001,1ST,PL,1ST PL,1,NE,1
3,002,2ND,AVE,2ND AVE,1,NW_SW_W,3
4,002,2ND,AVE,2ND AVE,2,CNTR_N_NE_S,4


In [28]:
test_agg['city_sector_group'].unique().shape

(32,)

In [29]:
test_agg.head()

,sort_order,ord_street_name,ord_street_type,ord_stname_type,group_id,city_sector_group,n_groups
0,001,1ST,AVE,1ST AVE,1,NW_SW_W,3
1,001,1ST,AVE,1ST AVE,2,CNTR_N_NE_S,4
2,001,1ST,PL,1ST PL,1,NE,1
3,002,2ND,AVE,2ND AVE,1,NW_SW_W,3
4,002,2ND,AVE,2ND AVE,2,CNTR_N_NE_S,4


In [30]:
# export to excel - this will be manually updated in conjunction with the plots
# generated below. This is necessary in order to better label street groups
# order the city sectors from top-to-bottom and left-to-right
city_sector_check = test_agg[['city_sector_group', 'ord_street_type', 'n_groups']].drop_duplicates().reset_index(drop = True)

In [31]:
city_sector_check.head()

,city_sector_group,ord_street_type,n_groups
0,NW_SW_W,AVE,3
1,CNTR_N_NE_S,AVE,4
2,NE,PL,1
3,SW,LN,1
4,SW,PL,1


In [32]:
city_sector_check.shape

(127, 3)

In [33]:
city_sector_check = city_sector_check.sort_values(by = ['city_sector_group', 'ord_street_type'])

In [34]:
ofpn = os.path.join(rc.ANALYSIS_OUTPUT_FILE_PATH, rc.S08_CPG_OUTPUT_FILE_NAME)

In [35]:
city_sector_check.to_excel(excel_writer=ofpn, index = False)

# join back to the working gdf

In [36]:
gdf.columns

Index(['f_intr_id', 't_intr_id', 'snd_id', 'snd_feacode', 'citycode',
       'stname_id', 'st_code', 'arterial_code', 'segment_type', 'agency_code',
       'access_code', 'divided_code', 'structure_type', 'legalloc_code',
       'vehicle_use_code', 'gis_seg_length', 'l_adrs_from', 'l_adrs_to',
       'r_adrs_from', 'r_adrs_to', 'ord_pre_dir', 'ord_street_name',
       'ord_street_type', 'ord_suf_dir', 'ord_stname_concat', 'l_city',
       'l_state', 'l_zip', 'r_city', 'r_state', 'r_zip', 'sndseg_update',
       'compkey', 'comptype', 'unitid', 'unitid2', 'city_sector',
       'ord_stname_unique', 'geometry', 'group_id'],
      dtype='object')

In [37]:
test_join = pd.merge(left = gdf, right = test_agg, how = 'outer', indicator=True)

In [38]:
test_join['_merge'].value_counts()

_merge
both          25773
left_only         0
right_only        0
Name: count, dtype: int64

In [39]:
test_join.shape

(25773, 45)

In [40]:
test_join.columns

Index(['f_intr_id', 't_intr_id', 'snd_id', 'snd_feacode', 'citycode',
       'stname_id', 'st_code', 'arterial_code', 'segment_type', 'agency_code',
       'access_code', 'divided_code', 'structure_type', 'legalloc_code',
       'vehicle_use_code', 'gis_seg_length', 'l_adrs_from', 'l_adrs_to',
       'r_adrs_from', 'r_adrs_to', 'ord_pre_dir', 'ord_street_name',
       'ord_street_type', 'ord_suf_dir', 'ord_stname_concat', 'l_city',
       'l_state', 'l_zip', 'r_city', 'r_state', 'r_zip', 'sndseg_update',
       'compkey', 'comptype', 'unitid', 'unitid2', 'city_sector',
       'ord_stname_unique', 'geometry', 'group_id', 'sort_order',
       'ord_stname_type', 'city_sector_group', 'n_groups', '_merge'],
      dtype='object')

In [41]:
gdf = test_join.drop(labels = '_merge', axis = 1)

# MAKE A PLOT OF THE DIFFERENT CITY GROUPS BY STREET TYPE

In [42]:
# use the same colors for each city sector
cdm = {}
dir_list = ['CNTR', 'E', 'N', 'NE', 'NW', 'S', 'SW', 'W']
for idir, dir_value in enumerate(dir_list):
    cdm[dir_value] = mpl.colormaps["Dark2"].colors[idir]

my_cmap = mpl.colors.ListedColormap([cdm[c] for c in dir_list])

In [43]:
wgdf = gdf.loc[gdf['n_groups'] > 1, :].copy()

In [44]:
wgdf.shape

(18184, 44)

In [45]:
col_names = ['city_sector_group', 'ord_street_type']
control_df = wgdf[col_names].drop_duplicates()

In [46]:
control_df.shape

(40, 2)

In [47]:
control_df.head()

,city_sector_group,ord_street_type
4,N_NE_NW,ST
73,S_SW,ST
126,N_NW,ST
144,N_NE,ST
561,NE_NW,ST


In [48]:
# plot bounds to use across each plot
bounds = [-122.4197794277490061,47.4803548409661005, -122.2200188105690017,47.7341482423694004]

In [49]:
# load the boundaries of the city sectors
ifpn = os.path.join(rc.OUTPUT_FILE_PATH, rc.S02_CITY_SECTORS_OUT_FILE_NAME)
cs_gdf = gpd.read_file(filename=ifpn)

In [50]:
control_df.head()

,city_sector_group,ord_street_type
4,N_NE_NW,ST
73,S_SW,ST
126,N_NW,ST
144,N_NE,ST
561,NE_NW,ST


In [51]:
ost = 'BR'
pgdf = gdf.loc[(gdf['ord_street_type'] == ost), :]

In [52]:
pgdf['city_sector_group'].unique()

array(['S', 'N', 'NW_W', 'CNTR', 'W', 'E', 'SW', 'E_NE'], dtype=object)

In [53]:
pgdf.head()

,f_intr_id,t_intr_id,snd_id,snd_feacode,citycode,stname_id,st_code,arterial_code,segment_type,agency_code,...,unitid,unitid2,city_sector,ord_stname_unique,geometry,group_id,sort_order,ord_stname_type,city_sector_group,n_groups
2947,47553,17931,38578,5,0,2870,0,1,1,1,...,00705,0820,S,16TH AV S BR,"LINESTRING (-122.31449 47.52947, -122.31456 47...",1,016,16TH AV S BR,S,1
14143,8947,8739,11489,5,1,2966,1,1,1,1,...,05254,0258,N,AURORA BR N,"LINESTRING (-122.34732 47.64855, -122.34732 47...",1,AURORA,AURORA BR,N,1
14144,9107,8947,16480,5,1,2966,0,1,1,1,...,05254,0258,N,AURORA BR N,"LINESTRING (-122.3473 47.64604, -122.34731 47....",1,AURORA,AURORA BR,N,1
14145,9358,9107,16776,5,1,2966,1,1,1,1,...,05254,0258,N,AURORA BR N,"LINESTRING (-122.34717 47.643, -122.34719 47.6...",1,AURORA,AURORA BR,N,1
14236,7743,7517,10647,5,1,2974,0,1,1,1,...,05315,0440,NW,BALLARD BR NW,"LINESTRING (-122.37624 47.66008, -122.37624 47...",1,BALLARD,BALLARD BR,NW_W,2


In [54]:
make_plots = True
if make_plots:
    for cr, crow in control_df.iterrows():
        cpg = crow['city_sector_group']
        ost = crow['ord_street_type']

        output_file_name = f"{ost}_{cpg}.png"
        ofpn = os.path.join(rc.S08_PLOT_OUTPUT_FILE_PATH_CITY_SECTOR_GROUPS, output_file_name)
        print(ofpn)

        # now, let's make a map...
        pgdf = gdf.loc[(gdf['city_sector_group'] == cpg) &
                    (gdf['ord_street_type'] == ost), :]
        to_draw = pgdf[['city_sector', 'geometry']].dissolve(by = 'city_sector', as_index = False)
        #to_draw['coords'] = to_draw['geometry'].map(lambda x: x.centroid.coords[0])
        fig = plt.figure(layout = 'constrained', figsize = (5, 10))
        gs = GridSpec(1,1, figure = fig, height_ratios = [1])
        ax1 = fig.add_subplot(gs[0,0])
        ax1.set_xlim(bounds[0], bounds[2])
        ax1.set_ylim(bounds[1], bounds[3])
        cs_gdf.plot(ax = ax1, column = 'city_sector', alpha = .2)
        to_draw.plot(ax = ax1, column = 'city_sector', cmap = my_cmap,  linewidth = 5, legend = True)

        #for irrow, row in to_draw.iterrows():    
        #    ax1.annotate(text=row['city_sector'], xy=row['coords'], fontsize = 16 )

        ax1.set_axis_off()
        
        my_title = f"Street Type: {ost} | City Sector Group {cpg}"
        plt.title(label = my_title)

        #plt.show()

        fig.savefig(fname = ofpn)
        plt.close()

../../../project/seattle_streets/print/city_sector_groups\ST_N_NE_NW.png
../../../project/seattle_streets/print/city_sector_groups\ST_S_SW.png
../../../project/seattle_streets/print/city_sector_groups\ST_N_NW.png
../../../project/seattle_streets/print/city_sector_groups\ST_N_NE.png
../../../project/seattle_streets/print/city_sector_groups\ST_NE_NW.png
../../../project/seattle_streets/print/city_sector_groups\AVE_NW_SW_W.png
../../../project/seattle_streets/print/city_sector_groups\AVE_CNTR_E_NE_S.png
../../../project/seattle_streets/print/city_sector_groups\AVE_CNTR_E_S.png
../../../project/seattle_streets/print/city_sector_groups\AVE_CNTR_N_NE_S.png
../../../project/seattle_streets/print/city_sector_groups\AVE_SW_W.png
../../../project/seattle_streets/print/city_sector_groups\AVE_E_NE_S.png
../../../project/seattle_streets/print/city_sector_groups\AVE_NE_S.png
../../../project/seattle_streets/print/city_sector_groups\PL_S_SW.png
../../../project/seattle_streets/print/city_sector_group